In [49]:
import os
from dotenv import load_dotenv
load_dotenv()
# SCOPES: Gmail read-only
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
token_path = os.getenv("token_path")
credentials_path = os.getenv("credentials_path")

In [ ]:
import pandas as pd
from dags.utils import decode_zip, extract_headers, decode_body 
path_1 = "C:/airflow-docker/data/15-08-2025-10-41.json.gz"
path_2 = "C:/airflow-docker/data/starred-19-08-2025-15-18.json.gz"
decompressed_data_1 = decode_zip(path_1)
decompressed_data_2 = decode_zip(path_2)

df =pd.DataFrame(decompressed_data_1)
df2 =pd.DataFrame(decompressed_data_2)

In [64]:
df["Sender"].value_counts()

Sender
glassdoor jobs                        94
linkedin job alerts                   31
                                      19
linkedin                              17
freelancercom                         11
naukri                                 8
labmentix hr                           5
jushey from instahyre                  4
zepp student purchase program          4
glassdoor                              4
microsoft azure team                   4
google                                 3
delhivery limited                      3
kaggle                                 3
sathwik methari                        3
code 360 by cn                         2
glassdoor community                    2
braintrust                             2
noreplymaxonnet via relay              2
hult international business school     2
leetcode                               1
adda247                                1
lenovo listens                         1
simplilearn webinars                   1
vega inte

In [62]:
#1.5 seconds faster for 250 rows of data
df[["Subject","Sender"]] = df["Payload"].apply(lambda x: pd.Series(extract_headers(x)))
df["Body"] = df["Payload"].apply(decode_body)
df.drop(["Payload"], axis=1, inplace=True)


df2[["Subject","Sender"]] = df2["Payload"].apply(lambda x: pd.Series(extract_headers(x)))
df2["Body"] = df2["Payload"].apply(decode_body)
df2.drop(["Payload"], axis=1, inplace=True)
df2["Important"] = 1
df2.head(5)

,Id,Subject,Sender,Body,Important
0,198a74e16ed2573a,your fomo store shipment awb 27722210317645 is...,delhivery limited,order cancelled delivered hey methari satwik y...,1
1,198a8c9b7f0cb680,hpcl 2025 cbt admit card passport size photograph,,dear candidate this has reference to your appl...,1
2,198aeae7ccd54d44,hpcl 2025 cbt admit card passport size photograph,,dear candidate this has reference to your appl...,1
3,198a710359a7ded4,hpcl 2025 cbt admit card,,dear candidate this has reference to your appl...,1
4,1989f70956b3f9f9,hpcl 2025 cbt admit card,,dear candidate this has reference to your appl...,1


In [5]:
from dags.utils import extract_headers, decode_body 
for payload in decompressed_data_1["Payload"][19:20]:
    subject, sender = extract_headers(payload)
    body_text = decode_body(payload)

    print(f"\n--- Email ---")
    print(f"From: {sender}")
    print(f"Subject: {subject}")
    print(f"Body:\n{body_text}")


--- Email ---
From: 
Subject: transaction failure for standing instruction on your icici bank debit card
Body:
icici bank online dear customer your payment of inr 11900 towards google play app purchase services with standing instruction xeasctukqj through your icici bank debit card 0647 could not be processed click here to manage your standing instructions sincerely icici bank limited this is a system generated email please do not reply to this email confidentiality information and disclaimer this email message and its attachments may contain confidential proprietary or legally privileged information and is intended solely for the use of the individual or entity to whom it is addressed if you have erroneously received this message please delete it immediately and notify the sender if you are not the intended recipient of the email message you should not disseminate distribute or copy this email email transmission cannot be guaranteed to be secure or errorfree as information could be i

In [9]:
import torch
from multiprocessing import cpu_count
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
print(cpu_count())
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

16
PyTorch version: 2.8.0+cu129
CUDA available: True


In [11]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
out = tokenizer(body_text,
                truncation=True,       
                max_length=512,        
                padding="max_length", 
                return_tensors="pt",
                return_length = True)

In [13]:
print(out["length"])

tensor([512])
